<a href="https://colab.research.google.com/github/GithubofRuZhang/Robust-Princing-Optimization/blob/main/Extending_the_Scope_of_Robust_Quadratic_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Extending the Scope of Robust Quadratic Optimization

## Semidefinite program
A semidefinite program (SDP) is an optimization problem of the form
$$
\begin{array}{ll}
\operatorname{minimize} & \operatorname{tr}(C X) \\
\text { subject to } & \operatorname{tr}\left(A_i X\right)=b_i, \quad i=1, \ldots, p \\
& X \succeq 0,
\end{array}
$$
where $\operatorname{tr}$ is the trace function, $X \in \mathcal{S}^n$ is the optimization variable and $C, A_1, \ldots, A_p \in \mathcal{S}^n$, and $b_1, \ldots, b_p \in \mathcal{R}$ are problem data, and $X \succeq 0$ is a matrix inequality. Here $\mathcal{S}^n$ denotes the set of $n$-by- $n$ symmetric matrices.

### Example

In [6]:
# Import packages.
import cvxpy as cp
import numpy as np

# Generate a random SDP.
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is 2.654347058555728
A solution X is
[[ 1.60805504 -0.59770125 -0.69575821]
 [-0.59770125  0.22228555  0.24689067]
 [-0.69575821  0.24689067  1.39679134]]


### Robust Price Optimization

$\max _{x \in \mathcal{X}} \min _{A \in \mathcal{C}_\lambda} x^{\top} A v(x) \Leftrightarrow \min _{x \in \mathcal{X}} \max _{Q \in \mathcal{C}_\lambda^{\prime}} v(x)^{\top} Q v(x)$,

where, $\mathbb{C}_\lambda^{\prime}:=\left\{Q=\hat{Q}+ U , \quad\|U\|_F \leq \lambda\right\}$.

$\begin{aligned} & \min _{x \in \chi} \operatorname{tr}(\hat{Q} W)+\lambda\|W\|_F \\
 & \begin{array}{l}x \in X \\
W \in \mathbb{R}^{N \times N}\end{array}\left[\begin{array}{cc}W & {\left[x, 1]^{\top}\right.} \\
{[x, 1]} & 1\end{array}\right] \succeq O_{(N+1) \times \left(N+1\right)} \\ & \end{aligned}$

In [26]:
import cvxpy as cp
import numpy as np

# Problem data
N = 3  # Dimension of the matrix W and vector x (with last element being 1)
P = 10  # Weight of the Frobenius norm term in the objective function
# Generate a random positive definite matrix for Q_hat
B = np.random.randn(N, N)
Q_hat = B.T @ B  # This guarantees Q_hat to be positive definite

# Define variables
W = cp.Variable((N, N), symmetric=True)  # Symmetric matrix variable W
x = cp.Variable(N-1)  # Vector variable x (without the last element)

# Construct the vector [x1,...,xN-1,1] where the last element is fixed at 1
x_complete = cp.hstack([x, np.array([1])])

# Reshape x_complete to be a column vector for concatenation
x_column = cp.reshape(x_complete, (N, 1))

# The matrix we need to be positive semidefinite
# has W in the top-left corner, and x_complete as the last column and last row.
# A = cp.bmat([
#     [W, x_column.T],
#     [x_column, 1]
# ])
A = cp.vstack([cp.hstack([W, cp.reshape(x_complete, (N, 1))]),
               cp.hstack([cp.reshape(x_complete, (1, N)), np.array([[1]])])])
# Define the objective function
objective = cp.Minimize(cp.trace(Q_hat @ W) + P * cp.norm(W, "fro"))

# Define the constraints
constraints = [A >> 0]  # The augmented matrix must be positive semidefinite

# Additional constraints for x can be added here if necessary
# For example, if x elements are non-negative
# marginal cost as the lower bounds for the price vector
c=10
constraints += [x >= c]

# Define and solve the problem
prob = cp.Problem(objective, constraints)
prob.solve()

# Print the results
print("The optimal value is", prob.value)
print("A solution W is")
print(W.value)
print("A solution x is")
print(x_complete.value[:N-1])  # This will print x with the last element being 1


The optimal value is 2564.787906082474
A solution W is
[[ 99.99976044  99.99995702   9.99956856]
 [ 99.99995702 100.00016011   9.99958224]
 [  9.99956856   9.99958224   0.99992331]]
A solution x is
[10.00000819 10.00001973]


## Second-order cone program
A second-order cone program (SOCP) is an optimization problem of the form
$$
\begin{array}{ll}
\operatorname{minimize} & f^T x \\
\text { subject to } & \left\|A_i x+b_i\right\|_2 \leq c_i^T x+d_i, \quad i=1, \ldots, m \\
& F x=g,
\end{array}
$$
where $x \in \mathcal{R}^n$ is the optimization variable and $f \in \mathcal{R}^n, A_i \in \mathcal{R}^{n_i \times n}, b_i \in \mathcal{R}^{n_i}, c_i \in \mathcal{R}^n$, $d_i \in \mathcal{R}, F \in \mathcal{R}^{p \times n}$, and $g \in \mathcal{R}^p$ are problem data.


### Example
An example of an SOCP is the robust linear program
$$
\begin{array}{ll}
\operatorname{minimize} & c^T x \\
\text { subject to } & \left(a_i+u_i\right)^T x \leq b_i \text { for all }\left\|u_i\right\|_2 \leq 1, \quad i=1, \ldots, m
\end{array}
$$
where the problem data $a_i$ are known within an $\ell_2$-norm ball of radius one. The robust linear program can be rewritten as the SOCP
$$
\begin{array}{ll}
\operatorname{minimize} & c^T x \\
\text { subject to } & a_i^T x+\|x\|_2 \leq b_i, \quad i=1, \ldots, m,
\end{array}
$$

When we solve a SOCP, in addition to a solution $x^{\star}$, we obtain a dual solution $\lambda_i^{\star}$ corresponding to each second-order cone constraint. A non-zero $\lambda_i^{\star}$ indicates that the constraint $\left\|A_i x+b_i\right\|_2 \leq c_i^T x+d_i$ holds with equality for $x^{\star}$ and suggests that changing $d_i$ would change the optimal value.

In [27]:
# Import packages.
import cvxpy as cp
import numpy as np

# Generate a random feasible SOCP.
m = 3
n = 10
p = 5
n_i = 5
np.random.seed(2)
f = np.random.randn(n)
A = []
b = []
c = []
d = []
x0 = np.random.randn(n)
for i in range(m):
    A.append(np.random.randn(n_i, n))
    b.append(np.random.randn(n_i))
    c.append(np.random.randn(n))
    d.append(np.linalg.norm(A[i] @ x0 + b, 2) - c[i].T @ x0)
F = np.random.randn(p, n)
g = F @ x0

# Define and solve the CVXPY problem.
x = cp.Variable(n)
# We use cp.SOC(t, x) to create the SOC constraint ||x||_2 <= t.
soc_constraints = [
      cp.SOC(c[i].T @ x + d[i], A[i] @ x + b[i]) for i in range(m)
]
prob = cp.Problem(cp.Minimize(f.T@x),
                  soc_constraints + [F @ x == g])
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution x is")
print(x.value)
for i in range(m):
    print("SOC constraint %i dual variable solution" % i)
    print(soc_constraints[i].dual_value)

The optimal value is -9.582695716266382
A solution x is
[ 1.40303325  2.4194569   1.69146656 -0.26922215  1.30825472 -0.70834842
  0.19313706  1.64153496  0.47698583  0.66581033]
SOC constraint 0 dual variable solution
[array([0.61662526]), array([[ 0.35370661],
       [-0.02327185],
       [ 0.04253095],
       [ 0.06243588],
       [ 0.49886837]])]
SOC constraint 1 dual variable solution
[array([0.35283078]), array([[-0.14301082],
       [ 0.16539699],
       [-0.22027817],
       [ 0.15440264],
       [ 0.06571645]])]
SOC constraint 2 dual variable solution
[array([0.86510445]), array([[-0.114638  ],
       [-0.449291  ],
       [ 0.37810251],
       [-0.6144058 ],
       [-0.11377797]])]
